In [ ]:
import numpy as np

from py_wake.examples.data.hornsrev1 import Hornsrev1Site, HornsrevV80
from py_wake.wind_turbines import WindTurbine, WindTurbines
from py_wake.wind_turbines.power_ct_functions import PowerCtTabular
from py_wake.literature.noj import Jensen_1983 as NOJ

import matplotlib.pyplot as plt
from matplotlib.colors import TwoSlopeNorm

# For IEEE publication looks of the figure
plt.rcParams.update({
    "font.family": "Times New Roman",
    "font.size": 14,
})

In [ ]:
# --- derating ---
def make_derated_turbine(base_turbine, derate_factor, ws_tab, name_suffix, air_density=1.225):
    """Return a new WindTurbine with reduced induction (wake relief)."""
    P_watt = base_turbine.power(ws_tab)
    Ct = base_turbine.ct(ws_tab)

    D = base_turbine.diameter()
    H = base_turbine.hub_height()
    A = np.pi * (D/2.0)**2
    rho = air_density
    denom = 0.5 * rho * A * np.maximum(ws_tab, 1e-6)**3

    Ct_clipped = np.clip(Ct, 0.0, 0.9999)
    a = 0.5 * (1.0 - np.sqrt(1.0 - Ct_clipped))

    # Using simple relations from actuator disk theory, we scale down the axial induction factor, which
    # directly reduces the thrust coefficient Ct (and power coefficient Cp) and thus the wake's strength
    a_dr = np.clip(derate_factor * a, 0.0, 0.5 - 1e-6)
    Ct_dr = 4.0 * a_dr * (1.0 - a_dr)
    Cp_dr = 4.0 * a_dr * (1.0 - a_dr)**2

    P_dr = Cp_dr * denom
    P_dr = np.where(P_watt <= 0.0, 0.0, P_dr)

    pcf = PowerCtTabular(ws_tab, P_dr, 'w', Ct_dr, method='linear')
    return WindTurbine(name=f"V80_{name_suffix}", diameter=D, hub_height=H, powerCtFunction=pcf)


# --- layout & grouping ---
def build_layout(base_turbine, nx=7, ny=3, sx=6, sy=4):
    """7x3 grid, spacing sx*D (x) by sy*D (y)."""
    D = base_turbine.diameter()
    dx, dy = sx * D, sy * D
    xg, yg = np.meshgrid(np.arange(nx) * dx, np.arange(ny) * dy)
    x, y = xg.ravel(), yg.ravel()
    return x, y, dx, dy, D

def groups_spanwise_rows(y, dy):
    """Return 0/1/2 for bottom/middle/top rows (exact 7/7/7)."""
    row_centers = np.array([0.0, dy, 2*dy])
    # nearest centerline in y
    grp = np.argmin(np.abs(y[:, None] - row_centers[None, :]), axis=1).astype(int)
    return grp

# --- evaluation ---
def evaluate_row_derate(row_factors, ws=10.0, wd=180.0):
    """
    Row-wise derating with 3 rows (bottom/middle/top).
    row_factors: [f_bottom, f_middle, f_top]
    Returns mean farm power over wd_list [W].
    """

    site = Hornsrev1Site(ti=0.08) # Representative offshore turbulence intensity at HR1
    base = HornsrevV80() # Vestas V80-2MW

    # Derated turbine types for each row
    ws_tab = np.arange(3, 26)
    wt_row = [make_derated_turbine(base, f, ws_tab, f"_row{i+1}")
              for i, f in enumerate(row_factors)]
    wt_multi = WindTurbines.from_WindTurbine_lst(wt_row)

    # Layout & row assignment
    x, y, dx, dy, D = build_layout(base, nx=7, ny=3, sx=6, sy=4)
    type_i = groups_spanwise_rows(y, dy)  # 0=bottom, 1=middle, 2=top

    wf_model = NOJ(site, wt_multi)

    sim_res = wf_model(x=x, y=y, type=type_i, ws=[ws], wd=[wd])
    farm_power_W = sim_res.Power.sum().values.item()

    return float(farm_power_W)

# --- visualise ---
def heatmap_bottom_two_rows(ws=10.0, wd=180.0, fmin=0.70, fmax=1.00, step=0.05,
                            cmap='RdBu_r', center_at_zero=True, show_marker=True):
    """
    Square heatmap of Delta in farm power [%] for first and middle row derating factors (f_t=1.0).
    - f_b/m/t stands for bottom, middle and top rows respectively
    - In this scenario, we do not have a reason to derate the third row for power maximisation as
    the wake from the third row does not impact any wind turbines behind it
    - center_at_zero=True uses a diverging norm centered at 0
    - cmap: 'RdBu_r' found to be the nicest
    """
    factors = np.arange(fmin, fmax + 1e-9, step)
    Z = np.zeros((len(factors), len(factors)))

    # Baseline
    P0 = evaluate_row_derate([1.0, 1.0, 1.0], ws=ws, wd=wd) / 1e6

    for i, fb in enumerate(factors):
        for j, fm in enumerate(factors):
            P = evaluate_row_derate([fb, fm, 1.0], ws=ws, wd=wd) / 1e6
            Z[i, j] = (P - P0) / P0 * 100.0  # Delta [%]

    # Optimum
    imax = np.unravel_index(np.argmax(Z), Z.shape)
    fb_opt, fm_opt, gain = factors[imax[0]], factors[imax[1]], float(Z[imax])
    print(f"fb_opt = {fb_opt}, fm_opt = {fm_opt}, gain = {gain}")

    # Plot
    fig, ax = plt.subplots(figsize=(6,6))
    if center_at_zero:
        norm = TwoSlopeNorm(vcenter=0.0, vmin=Z.min(), vmax=Z.max())
    else:
        norm = None

    im = ax.imshow(
        Z, origin='lower', extent=[fmin, fmax, fmin, fmax],
        cmap=cmap, norm=norm
    )
    ax.set_aspect('equal')

    ax.set_ylim(fmin, fmax)
    #ax.invert_yaxis()  # Optional

    # Labels/titles
    ax.set_xlabel("Middle row derating factor [-]")
    ax.set_ylabel("First row derating factor [-]")
    #ax.set_title(f"(ws={ws} m/s, wd={wd}°)") # Optional title

    # Colorbar sized for square plot
    cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_label("Δ Farm power [%] vs baseline")

    # Baseline / optimum markers
    ax.scatter([0.999], [0.999], color='red', marker='x', s=90, label="Baseline")
    if show_marker:
        ax.scatter([fm_opt], [fb_opt], color='white', edgecolors='black', s=60, zorder=3, label="Optimum")
    ax.legend(loc='upper left')

    fig.tight_layout()
    plt.savefig("fig5.png", dpi=600, bbox_inches="tight") # Save with publication-level quality
    plt.show()

    return factors, Z, (fb_opt, fm_opt, gain)

In [ ]:
factors_no_derating = [1.0, 1.0, 1.0]  # no derating, 26.25 MW @ 10m/s
P_baseline = evaluate_row_derate(factors_no_derating, ws=10.0, wd=180.0)
factors_1 = [0.95, 1.0, 1.0]  # first row derating, 26.33 MW @ 10m/s
P_1 = evaluate_row_derate(factors_1, ws=10.0, wd=180.0)
factors_2 = [1.0, 0.95, 1.0] # second row derating, 26.35 MW @ 10m/s
P_2 = evaluate_row_derate(factors_2, ws=10.0, wd=180.0)
factors_3 = [1.0, 1.0, 0.95] # third row derating, 26.16 MW @ 10m/s
P_3= evaluate_row_derate(factors_3, ws=10.0, wd=180.0)

print(f"Baseline farm power:           {(P_baseline / 1e6):.4g} [MW]")
print(f"First row derated farm power:  {(P_1 / 1e6):.4g} [MW]") # Should be higher than baseline.
print(f"Second row derated farm power: {(P_2 / 1e6):.4g} [MW]") # Should be higher than baseline.
print(f"Third row derated farm power:  {(P_3 / 1e6):.4g} [MW]") # Should be lower than baseline.

In [ ]:
heatmap_bottom_two_rows(ws=10.0, wd=180.0, fmin=0.70, fmax=1.00, step=0.01)